In [1]:

%load_ext sql

In [2]:
%sql postgresql://localhost/upreads_dev

In [7]:
%%sql
drop table users

 * postgresql://localhost/upreads_dev
Done.


[]

In [6]:
%%sql
drop table shelves

 * postgresql://localhost/upreads_dev
Done.


[]

In [5]:
%%sql
drop table user_shelves

 * postgresql://localhost/upreads_dev
Done.


[]

In [4]:
%%sql
drop table user_books

 * postgresql://localhost/upreads_dev
Done.


[]

In [3]:
%%sql
drop table shelf_books

 * postgresql://localhost/upreads_dev
Done.


[]

In [12]:
%%sql
CREATE TABLE IF NOT EXISTS users (
    id SERIAL PRIMARY KEY,
    username VARCHAR(255) UNIQUE NOT NULL,
    email VARCHAR(255) UNIQUE NOT NULL,
    password_digest TEXT NOT NULL
);

 * postgresql://localhost/upreads_dev
Done.


[]

In [13]:
%%sql 
CREATE TABLE IF NOT EXISTS shelves(
    id SERIAL PRIMARY KEY,
    is_public BOOLEAN,
    creator_user_id INTEGER,
    shelf_name VARCHAR(255)
);

 * postgresql://localhost/upreads_dev
Done.


[]

In [14]:
%%sql 
CREATE TABLE IF NOT EXISTS user_shelves(
    id SERIAL PRIMARY KEY,
    user_id INTEGER,
    shelf_id INTEGER REFERENCES shelves(id) UNIQUE
);

 * postgresql://localhost/upreads_dev
Done.


[]

In [15]:
%%sql 
CREATE TABLE IF NOT EXISTS user_books(
    id SERIAL PRIMARY KEY,
    user_id INTEGER,
    google_book_id VARCHAR(255),
    status VARCHAR(255),
    shelf_id INTEGER REFERENCES user_shelves(shelf_id),
    title VARCHAR(255),
    author VARCHAR(255),
    cover_img VARCHAR(255)
);

 * postgresql://localhost/upreads_dev
Done.


[]

In [16]:
%%sql
CREATE TABLE IF NOT EXISTS shelf_books(
    id SERIAL PRIMARY KEY,
    shelf_id INTEGER,
    google_book_id VARCHAR(255),
    title VARCHAR(255),
    author VARCHAR(255),
    cover_img VARCHAR(255)
);   

 * postgresql://localhost/upreads_dev
Done.


[]

In [17]:
%%sql 
INSERT INTO shelves
    (is_public, creator_user_id, shelf_name)
VALUES
    (true, 1, 'Black Lives Matter'),
    (true, 1, 'Black Women Authors, Fiction')
;    

 * postgresql://localhost/upreads_dev
2 rows affected.


[]

In [18]:
%%sql
INSERT INTO user_shelves
    (user_id, shelf_id)
VALUES
    (1, 1),
    (1, 2)
;

 * postgresql://localhost/upreads_dev
2 rows affected.


[]

In [20]:
%%sql
INSERT INTO shelf_books
    (shelf_id, google_book_id, title, author, cover_img)
VALUES
    (1, 'REKIbk12HxkC', 'Point Blank', 'Anthony Horowitz', 'http://books.google.com/books/content?id=REKIbk12HxkC&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api'),
    (1, 'cYzzDwAAQBAJ', 'Alex Rider: Secret Weapon', 'Anthony Horowitz', 'http://books.google.com/books/content?id=cYzzDwAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api' ),
    (2, 'vMcBnbBt68o', 'Nightshade', 'Anthony Horowitz', 'http://books.google.com/books/content?id=wSSZDwAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api')
;

 * postgresql://localhost/upreads_dev
3 rows affected.


[]

In [21]:
%%sql
INSERT INTO user_books
    (user_id, status, google_book_id, shelf_id, title, author, cover_img)
VALUES
    (1, 'unread', 'REKIbk12HxkC', 1, 'Point Blank', 'Anthony Horowitz', 'http://books.google.com/books/content?id=REKIbk12HxkC&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api'),
    (1, 'unread','cYzzDwAAQBAJ', 1, 'Alex Rider: Secret Weapon', 'Anthony Horowitz', 'http://books.google.com/books/content?id=cYzzDwAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api' ),
    (1, 'unread', 'vMcBnbBt68o', 2, 'Nightshade', 'Anthony Horowitz', 'http://books.google.com/books/content?id=wSSZDwAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api')
    ;

 * postgresql://localhost/upreads_dev
3 rows affected.


[]

## Get user books
- get all books that a user has saved by shelf

In [22]:
%%sql
select * from user_books where user_id = 1;

 * postgresql://localhost/upreads_dev
3 rows affected.


id,user_id,google_book_id,status,shelf_id,title,author,cover_img
1,1,REKIbk12HxkC,unread,1,Point Blank,Anthony Horowitz,http://books.google.com/books/content?id=REKIbk12HxkC&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api
2,1,cYzzDwAAQBAJ,unread,1,Alex Rider: Secret Weapon,Anthony Horowitz,http://books.google.com/books/content?id=cYzzDwAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api
3,1,vMcBnbBt68o,unread,2,Nightshade,Anthony Horowitz,http://books.google.com/books/content?id=wSSZDwAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api


In [4]:
%%sql
SELECT
    shelf_name,
    user_shelves.shelf_id,
    google_book_id
FROM
    (
        SELECT
            shelf_name,
            us.shelf_id
        FROM user_shelves us
        LEFT JOIN shelves on us.shelf_id = shelves.id
        WHERE us.user_id = 1
    ) user_shelves
LEFT JOIN shelf_books on user_shelves.shelf_id = shelf_books.shelf_id


 * postgresql://localhost/upreads_dev
5 rows affected.


shelf_name,shelf_id,google_book_id
Black Lives Matter,1,1BLM
Black Lives Matter,1,2BLM
Black Lives Matter,1,3BLM
"Black Women Authors, Fiction",2,1WOMEN
"Black Women Authors, Fiction",2,2WOMEN


In [7]:
%%sql
SELECT
    shelf_name,
    shelves.id,
    google_book_id
FROM shelves
LEFT JOIN
    shelf_books on shelves.id = shelf_books.shelf_id
WHERE is_public = true
;



 * postgresql://localhost/upreads_dev
5 rows affected.


shelf_name,id,google_book_id,is_public
Black Lives Matter,1,1BLM,True
Black Lives Matter,1,2BLM,True
Black Lives Matter,1,3BLM,True
"Black Women Authors, Fiction",2,1WOMEN,True
"Black Women Authors, Fiction",2,2WOMEN,True


In [8]:
%%sql
select * from user_shelves;

 * postgresql://localhost/upreads_dev
2 rows affected.


id,user_id,shelf_id
1,1,1
2,1,2


In [21]:
%%sql
select * from user_books;

 * postgresql://localhost/upreads_dev
5 rows affected.


id,user_id,google_book_id,status,shelf_id
1,1,p7uGzQEACAAJ,unread,1
2,1,FylCUanlGhAC,read,1
3,1,ltClDwAAQBAJ,unread,1
4,1,r6c8DAAAQBAJ,unread,2
5,1,5GbdTc9OJ78C,read,1
